In [1]:
import billboard


In [33]:
chart = billboard.ChartData('hot-100')
# print(chart)

In [34]:
artists = []
songs = []

for song in chart:
    songs.append(song.title)
    artists.append(song.artist)
# print(artists)
# print()
# print(songs)

In [35]:
chart = billboard.ChartData('billboard-200')
print(chart)

billboard-200 chart from 2018-10-13
-----------------------------------
1. 'Tha Carter V' by Lil Wayne
2. 'YSIV' by Logic
3. 'Dancing Queen' by Cher
4. 'Luca Brasi 3' by Kevin Gates
5. 'Scorpion' by Drake
6. 'Kamikaze' by Eminem
7. 'ASTROWORLD' by Travis Scott
8. 'beerbongs & bentleys' by Post Malone
9. 'An American Treasure' by Tom Petty
10. '?' by XXXTENTACION
11. 'Goodbye & Good Riddance' by Juice WRLD
12. 'Sweetener' by Ariana Grande
13. 'Queen' by Nicki Minaj
14. 'Invasion Of Privacy' by Cardi B
15. 'Look Up Child' by Lauren Daigle
16. 'The Greatest Showman' by Soundtrack
17. 'This One's For You' by Luke Combs
18. 'Cry Pretty' by Carrie Underwood
19. 'East Atlanta Love Letter' by 6LACK
20. 'On The Rvn!! (EP)' by Young Thug
21. '÷ (Divide)' by Ed Sheeran
22. 'Harder Than Ever' by Lil Baby
23. 'Stoney' by Post Malone
24. 'Love Yourself: Answer' by BTS
25. 'Hallelujah Here Below' by Elevation Worship
26. 'Swimming' by Mac Miller
27. '17' by XXXTENTACION
28. 'Bridges' by Josh Groban
2